In [8]:
from MapXtremePCIC import MapXtremePCIC

In [9]:
PATH = './data/pr_24hr_50yrs/'
MapXtreme = MapXtremePCIC('110m', 'eof', PATH, 'pr')

In [3]:
MapXtreme.read_data()

<xarray.Dataset>
Dimensions:  (rlat: 124, rlon: 149, run: 35, x: 124, y: 149)
Coordinates:
    lon      (x, y) float64 233.5 233.8 234.2 234.6 ... 331.5 332.0 332.5 333.0
    lat      (x, y) float64 14.04 14.2 14.36 14.51 ... 60.17 59.87 59.56 59.26
  * rlon     (rlon) float64 -32.56 -32.12 -31.68 -31.24 ... 31.68 32.12 32.56
  * rlat     (rlat) float64 -27.28 -26.84 -26.4 -25.96 ... 25.96 26.4 26.84
Dimensions without coordinates: run, x, y
Data variables:
    obs      (x, y, run) float64 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    obs:      mm h-1
    lon:      degrees
    lat:      degrees
    rlon:     degrees
    rlat:     degrees

In [4]:
MapXtreme.get_arr()

(35, 18204, array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]))

In [11]:
MapXtreme.get_df().dropna().sample(frac=0.01)

,areas,rlat,rlon,run0,run1,run2,run3,run4,run5,run6,...,run25,run26,run27,run28,run29,run30,run31,run32,run33,run34
6306,2325.166695,-8.799998,7.040009,5.477463,5.823375,5.633585,5.627803,5.849770,5.781929,5.813034,...,6.753105,5.324561,5.300463,5.669088,5.679893,6.132356,5.587154,5.819047,5.607723,6.283651
15904,2217.432487,19.800001,-2.639999,2.319510,2.402132,2.202994,2.137139,2.206352,2.246877,2.552114,...,2.649207,2.694425,2.280882,2.501273,2.459148,2.346518,2.496009,2.364824,2.436273,2.678594
8368,2375.588478,-2.639999,2.639984,5.477764,5.630385,5.695490,5.683086,5.144169,5.078913,5.454853,...,6.022448,5.663394,6.179428,5.628813,4.978403,5.439196,5.946013,6.608844,5.374068,5.445987
8357,2379.167648,-2.639999,-2.199997,4.578641,5.682903,5.410180,4.504844,4.768818,5.072014,5.055875,...,5.488507,5.070752,5.432767,5.735085,5.107081,5.548853,4.656959,5.684996,4.969947,5.031546
3340,2265.841090,-17.599997,4.399994,6.779444,7.134372,6.380640,6.843326,7.004902,8.188025,6.247184,...,7.947843,6.242516,6.164166,7.171173,6.486387,8.137124,6.974116,7.254761,7.711489,7.497353
5838,2342.849099,-10.119999,-3.520004,5.898898,6.485762,5.221248,5.948037,5.867398,5.556581,4.937887,...,6.126651,5.389482,6.563198,5.530765,5.470182,6.262217,5.642726,6.197353,5.454193,5.919817
14249,1995.334289,14.960001,-14.520004,4.149787,3.625259,3.919579,4.593092,3.547873,3.881557,4.288186,...,3.983203,3.951564,3.650504,3.968894,3.934224,4.182928,4.370202,4.140370,3.749808,3.788735
6003,2341.773324,-9.679998,3.960007,6.573605,6.712597,6.110299,6.632106,6.815580,6.506184,6.591817,...,6.843470,6.595451,5.841254,7.100310,6.541664,6.120274,6.159525,5.972023,6.932149,6.731382
17069,1858.500189,23.320002,-11.000000,2.017261,1.914407,1.895084,2.050880,2.009852,2.276119,1.934477,...,2.067334,1.746199,1.882064,1.982766,2.141558,1.830820,1.919241,1.971861,1.979035,1.962788
18084,1232.322808,26.400002,-20.239990,1.811596,1.686760,1.692753,1.813070,2.060256,1.900973,1.778335,...,1.813135,1.652566,1.915070,1.989561,1.771439,1.843628,2.057605,1.818253,2.028202,1.906172
